# Using Identity Links Deep Neural Networks

Shallow Deep neural networks are the 'defacto' model for structured data consisting only of numeric data (e.g., sensor measurements).

When numeric data columns are not normalized within the same range, we find that many times a deep neural network will either not converge, or overfit to the training data. We will demonstrate that a shallow deep neural network can be trained to converge without overfitting on non-normalized numeric data by adding an identity link from the input vector to each shallow layer.

We observe from this technique the following:

    1. Introduces regularization into the model (preventing overfitting).
    2. Adds stability to the slope of the valuation loss/accuracy with non-normalized data.
    
While the identity link adds some parameters, we add it as a concatenation vector operation. This is a fast operation, and only adds a nominal number of parameters at each layer.

## Datasets

In this notebook, we use two well-known numeric only datasets: 'iris' and 'wine' for demonstration. Both datasets consists of numeric data only with three output classes. 

### Iris Dataset

This dataset consists of 150 examples for three output classes (50 each). The data consists of 4 numeric columns.

See (UCI Machine Learning Repository for more details)[https://archive.ics.uci.edu/ml/datasets/iris].

In [31]:
from sklearn import datasets
iris = datasets.load_iris()
X_iris = iris.data
Y_iris = iris.target

### Wine Dataset

This dataset consists of 178 examples for three output classes (~60 each). The data consists of 13 numeric columns.

See (UCI Machine Learning Repository for more details)[https://archive.ics.uci.edu/ml/datasets/wine].

In [47]:
wine = datasets.load_wine()
X_wine = wine.data
Y_wine = wine.target

## Models

For both datasets, we will use the same shallow deep model, with the only difference being the input vector:

        input layer : 10 node dense layer
        hidden layer: 10 node dense layer
        output layer: 3 node dense layer
        
We will train using non-normalized data on two versions of the model for each dataset. In the first version, we will train w/o the identity linkand then with the identity link.

### Iris Model

    iris_model  : model w/o identity link
    iris_modelx : model with identity link

In [39]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Concatenate

inputs = Input(shape=(4,))
x = Dense(10, activation='relu')(inputs)
x = Dense(10, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
iris_model = Model(inputs, outputs)
iris_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
iris_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_30 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_31 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_32 (Dense)             (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [40]:
inputs = Input(shape=(4,))
x = Dense(10, activation='relu')(inputs)
x = Concatenate()([inputs, x])
x = Dense(10, activation='relu')(x)
x = Concatenate()([inputs, x])
outputs = Dense(3, activation='softmax')(x)
iris_modelx = Model(inputs, outputs)
iris_modelx.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
iris_modelx.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 10)           50          input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_10 (Concatenate)    (None, 14)           0           input_12[0][0]                   
                                                                 dense_33[0][0]                   
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 10)           150         concatenate_10[0][0]      

### Wine Model

    wine_model  : model w/o identity link
    wine_modelx : model with identity link

In [41]:
inputs = Input(shape=(13,))
x = Dense(10, activation='relu')(inputs)
x = Dense(10, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
wine_model = Model(inputs, outputs)
wine_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
wine_model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 13)]              0         
_________________________________________________________________
dense_36 (Dense)             (None, 10)                140       
_________________________________________________________________
dense_37 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_38 (Dense)             (None, 3)                 33        
Total params: 283
Trainable params: 283
Non-trainable params: 0
_________________________________________________________________


In [42]:
inputs = Input(shape=(13,))
x = Dense(10, activation='relu')(inputs)
x = Concatenate()([inputs, x])
x = Dense(10, activation='relu')(x)
x = Concatenate()([inputs, x])
outputs = Dense(3, activation='softmax')(x)
wine_modelx = Model(inputs, outputs)
wine_modelx.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
wine_modelx.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense_39 (Dense)                (None, 10)           140         input_14[0][0]                   
__________________________________________________________________________________________________
concatenate_12 (Concatenate)    (None, 23)           0           input_14[0][0]                   
                                                                 dense_39[0][0]                   
__________________________________________________________________________________________________
dense_40 (Dense)                (None, 10)           240         concatenate_12[0][0]      

## Results

Each traing session will train for 30 epochs for stocastic gradient descent (batch=1).

### Iris w/o identity Link

In [44]:
 iris_model.fit(X_iris, Y_iris, epochs=30, batch_size=1, verbose=1, validation_split=0.2)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 2ms/sample - loss: 2.5674 - acc: 0.1667 - val_loss: 0.4745 - val_acc: 1.0000
Epoch 2/30
120/120 [==============================] - 0s 729us/sample - loss: 0.8648 - acc: 0.7500 - val_loss: 1.0285 - val_acc: 0.0000e+00
Epoch 3/30
120/120 [==============================] - 0s 723us/sample - loss: 0.6682 - acc: 0.8500 - val_loss: 0.8579 - val_acc: 0.0000e+00
Epoch 4/30
120/120 [==============================] - 0s 736us/sample - loss: 0.5293 - acc: 0.8833 - val_loss: 0.7385 - val_acc: 0.3667
Epoch 5/30
120/120 [==============================] - 0s 717us/sample - loss: 0.4477 - acc: 0.8917 - val_loss: 0.9266 - val_acc: 0.0000e+00
Epoch 6/30
120/120 [==============================] - 0s 797us/sample - loss: 0.3962 - acc: 0.8667 - val_loss: 0.6592 - val_acc: 0.6000
Epoch 7/30
120/120 [==============================] - 0s 759us/sample - loss: 0.3554 - acc: 0.8833 - val_loss: 0.6822 - val_acc: 

In [45]:
iris_modelx.fit(X_iris, Y_iris, epochs=30, batch_size=1, verbose=1, validation_split=0.2)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 2ms/sample - loss: 1.2748 - acc: 0.3000 - val_loss: 1.3369 - val_acc: 0.0000e+00
Epoch 2/30
120/120 [==============================] - 0s 840us/sample - loss: 0.7657 - acc: 0.7750 - val_loss: 0.9735 - val_acc: 0.0000e+00
Epoch 3/30
120/120 [==============================] - 0s 771us/sample - loss: 0.5576 - acc: 0.8417 - val_loss: 1.0936 - val_acc: 0.0000e+00
Epoch 4/30
120/120 [==============================] - 0s 750us/sample - loss: 0.4395 - acc: 0.8500 - val_loss: 1.3736 - val_acc: 0.0000e+00
Epoch 5/30
120/120 [==============================] - 0s 772us/sample - loss: 0.3848 - acc: 0.8667 - val_loss: 1.2372 - val_acc: 0.0000e+00
Epoch 6/30
120/120 [==============================] - 0s 796us/sample - loss: 0.3519 - acc: 0.8583 - val_loss: 0.5310 - val_acc: 0.9333
Epoch 7/30
120/120 [==============================] - 0s 783us/sample - loss: 0.3227 - acc: 0.9000 - val_loss: 0.8014 - v

### Wine Results

In [48]:
wine_model.fit(X_wine, Y_wine, epochs=30, batch_size=1, verbose=1, validation_split=0.2)

Train on 142 samples, validate on 36 samples
Epoch 1/30
142/142 [==============================] - 0s 2ms/sample - loss: 55.8233 - acc: 0.5282 - val_loss: 79.0462 - val_acc: 0.0000e+00
Epoch 2/30
142/142 [==============================] - 0s 785us/sample - loss: 4.7335 - acc: 0.7817 - val_loss: 23.1832 - val_acc: 0.0000e+00
Epoch 3/30
142/142 [==============================] - 0s 762us/sample - loss: 1.9010 - acc: 0.7676 - val_loss: 8.2698 - val_acc: 0.0278
Epoch 4/30
142/142 [==============================] - 0s 756us/sample - loss: 1.6688 - acc: 0.7254 - val_loss: 7.8907 - val_acc: 0.0000e+00
Epoch 5/30
142/142 [==============================] - 0s 755us/sample - loss: 1.5295 - acc: 0.7254 - val_loss: 5.2020 - val_acc: 0.2500
Epoch 6/30
142/142 [==============================] - 0s 755us/sample - loss: 1.2432 - acc: 0.7606 - val_loss: 2.5615 - val_acc: 0.0000e+00
Epoch 7/30
142/142 [==============================] - 0s 760us/sample - loss: 1.3083 - acc: 0.7465 - val_loss: 5.7311 - va

In [49]:
wine_modelx.fit(X, Y, epochs=30, batch_size=1, verbose=1, validation_split=0.1)

Train on 160 samples, validate on 18 samples
Epoch 1/30
160/160 [==============================] - 0s 2ms/sample - loss: 79.8944 - acc: 0.1937 - val_loss: 9.2984 - val_acc: 0.0000e+00
Epoch 2/30
160/160 [==============================] - 0s 750us/sample - loss: 6.9795 - acc: 0.3750 - val_loss: 24.9290 - val_acc: 0.0000e+00
Epoch 3/30
160/160 [==============================] - 0s 747us/sample - loss: 3.9265 - acc: 0.5688 - val_loss: 1.7274 - val_acc: 0.5000
Epoch 4/30
160/160 [==============================] - 0s 744us/sample - loss: 2.4195 - acc: 0.6313 - val_loss: 0.8600 - val_acc: 0.6111
Epoch 5/30
160/160 [==============================] - 0s 751us/sample - loss: 1.7023 - acc: 0.6812 - val_loss: 2.2329 - val_acc: 0.1111
Epoch 6/30
160/160 [==============================] - 0s 738us/sample - loss: 1.4491 - acc: 0.6812 - val_loss: 0.0116 - val_acc: 1.0000
Epoch 7/30
160/160 [==============================] - 0s 738us/sample - loss: 1.4903 - acc: 0.6812 - val_loss: 2.6650 - val_acc: 0.